In [1]:
# This notebook uses Long Short-Term Memory (LSTM) to predict the closing stock price of a corporation using the past 60 day stock price.

# Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
plt.style.use('fivethirtyeight')

/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the past year stock price
# today's date
today = pd.to_datetime('today').strftime('%Y-%m-%d')
# 5 year ago
start = pd.to_datetime('today') - pd.DateOffset(years=5)
df = web.DataReader('AAPL', data_source='yahoo', start=start, end=today)

In [3]:
# Create a new dataframe with only the 'Close' column
data = df.filter(['Close'])
# Convert the dataframe to a numpy array
dataset = data.values
# Get the number of rows to train the model on
training_data_len = math.ceil( len(dataset) * .8 )


In [4]:
# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [5]:
# Create the training data set
train_data = scaled_data[0:training_data_len  , : ]
# Split the data into x_train and y_train data sets
x_train=[]
y_train = []

for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

In [6]:
# Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))


In [7]:
# Build the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=50, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [8]:
# Create the model
model = LSTM()

# Define the loss and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
for seq,labels in zip(x_train,y_train):
    seq.view

(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 1)


In [9]:
# train the model
epochs = 100
for i in range(epochs):
    for seq, labels in zip(x_train, y_train):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

TypeError: view() takes from 0 to 2 positional arguments but 3 were given